# Reinforcement Learning for Quantum Circuit Design

In this notebook, we introduce Q-learning for quantum circuit design.

1. [Introduction](#intro)
2. [Target Gates](#gates)
3. [Environment](#gym)
4. [Agent](#agent)
5. [Training](#train)
6. [Testing](#test)

<a id='intro'></a>
## 1. Introduction

Q-learning is a  reinforcement learning algorithm that learns the optimal action policy for an agent interacting with an environment. It operates by estimating Q-values (expected future rewards of state-action pairs) by taking a specific action in a given state and following the optimal policy. It updates these values based on the rewards received and uses the Bellman equation to converge to the optimal policy that maximizes long-term rewards.

Using Q-learning, we want an automatic and scalable approach for the circuit design of complex quantum circuits into simpler, implementable gates on quantum hardware.

### Imports
gym: reinforcement learning toolkit \
numpy: array manipulation \
qiskit: quantum circuits \
qiskit-aer: circuit simulation

In [ ]:
%pip install gym
%pip install numpy
%pip install qiskit
%pip install qiskit_aer

In [1]:
import gym
from gym import spaces
import hashlib
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import HGate, CXGate, SGate, TGate, XGate, YGate, ZGate
from qiskit.quantum_info import Operator

<a id='gates'></a>
## 2. Target Gates

These are the target circuits we want to eventually construct through smaller gates. Each circuit state can be represented as a unitary matrix $U$.

$H = \frac{1}{\sqrt{2}}\begin{pmatrix}1 & 1 \\ 1 & -1\end{pmatrix}$

$\text{CNOT} =  \begin{pmatrix}
  1 & 0 & 0 & 0 \\
  0 & 1 & 0 & 0 \\
  0 & 0 & 0 & 1 \\
  0 & 0 & 1 & 0 
  \end{pmatrix}$

$\ket{\Phi^{+}} = \text{CNOT} \cdot (H \otimes I) = 
\begin{pmatrix}
  1 & 0 & 0 & 0 \\
  0 & 1 & 0 & 0 \\
  0 & 0 & 0 & 1 \\
  0 & 0 & 1 & 0 
  \end{pmatrix} \cdot
  (\frac{1}{\sqrt{2}}\begin{pmatrix}1 & 1 \\ 1 & -1\end{pmatrix} \otimes \begin{pmatrix}1 & 0 \\ 0 & 1\end{pmatrix}) =
\frac{1}{\sqrt{2}}\begin{pmatrix}
  1 & 0 & 1 & 0 \\
  0 & 1 & 0 & 1 \\
  0 & 1 & 0 & -1 \\
  1 & 0 & -1 & 0 
  \end{pmatrix}$

$CZ =  \begin{pmatrix}
  1 & 0 & 0 & 0 \\
  0 & 1 & 0 & 0 \\
  0 & 0 & 1 & 0 \\
  0 & 0 & 0 & -1 
  \end{pmatrix}$

$\text{SWAP} =  \begin{pmatrix}
  1 & 0 & 0 & 0 \\
  0 & 0 & 1 & 0 \\
  0 & 1 & 0 & 0 \\
  0 & 0 & 0 & 1 
  \end{pmatrix}$

$\text{iSWAP} =  \begin{pmatrix}
  1 & 0 & 0 & 0 \\
  0 & 0 & 1j & 0 \\
  0 & 1j & 0 & 0 \\
  0 & 0 & 0 & 1 
  \end{pmatrix}$

In [2]:
H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)

CNOT = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

bell_state_unitary = Operator(CNOT) @ Operator(np.kron(H, np.eye(2)))

cz_matrix = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, -1]
])

swap_matrix = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1]
])

iswap_matrix = np.array([
    [1, 0, 0, 0],
    [0, 0, 1j, 0],
    [0, 1j, 0, 0],
    [0, 0, 0, 1]
])

<a id='gym'></a>
## 3. Environment

We use the Python library **gym** to define the environment for our Q-learning algorithm. This environment represents a problem or task that the **agent** needs to solve or optimize. It provides feedback to the agent based on the agent's actions.

In our environment, we set the target unitary that we want to construct to be the Bell state $\ket{\Phi^{+}}$, shown below, as well as the number of qubits needed. We also initialize a quantum circuit with the number of qubits.

<img src="images/phi_plus.png" alt="Image 1" width="700" height="500" style="display: inline-block; margin: 10px;">

We predefine the **observation** (or state) and **action** spaces for our agent. \
There are $100$ possible discrete states the quantum circuit can take. In our example, states are converted into a limited integer range of indices through the hash function to associate them with the current circuit. Thus, each state represents a quantum circuit. \
There are $14$ possible discrete actions the agent can choose from, defined in the following **Agent** class: $H$ gate,  $\text{CNOT}$ gate, $S$ gate, $T$ gate, $X$ gate, $Y$ gate, and $Z$ gate.

In [3]:
class QuantumEnv(gym.Env):
    def __init__(self):
        super(QuantumEnv, self).__init__()
        
        # Set number of qubits for our target unitary
        self.num_qubits = 2
        # Initialize quantum circuit with number of qubits
        self.circuit = QuantumCircuit(self.num_qubits)
        
        self.target_unitary = bell_state_unitary # Change target circuit (bell_state,cz,swap,iswap)

        # Define action and observation space
        self.action_space = spaces.Discrete(14)  # Number of possible actions
        self.observation_space = spaces.Discrete(100)  # Number of possible states (hashes)

        # Store hashed circuits
        self.state_to_index = {}
        self.index_to_state = []

    # Hash function
    def _hash_circuit(self, circuit: QuantumCircuit) -> str:
        circuit_str = circuit.draw(output='text').__str__()
        circuit_hash = hashlib.sha256(circuit_str.encode('utf-8')).hexdigest()
        hash_int = int(circuit_hash, 16)
        return hash_int % 100

    # Retrieve the index of our current state
    def get_state_index(self, state: QuantumCircuit) -> int:
        state_hash = self._hash_circuit(state)
        if state_hash not in self.state_to_index:
            index = len(self.state_to_index)
            self.state_to_index[state_hash] = index
            self.index_to_state.append(state_hash)
        return self.state_to_index[state_hash]

    # Retrieve our current state
    def get_state_from_index(self, index: int) -> QuantumCircuit:
        state_hash = self.index_to_state[index]
        for circuit_hash, idx in self.state_to_index.items():
            if idx == index:
                return self._hash_circuit(circuit_hash)
        return None

    # Resets environment and return sto initial state
    def reset(self):
        self.circuit = QuantumCircuit(self.num_qubits)
        return self.get_state_index(self.circuit)

    # Takes an action, advances the environment, and returns the next state, reward, and done flag.
    def step(self, action, qubits):
        # Execute the action and return next_state, reward, done
        self.circuit.append(action, qubits)
        state_index = self.get_state_index(self.circuit)
        reward, done = self._reward(self.target_unitary)
        return state_index, reward, done

    # Visualizes current state
    def render(self):
        print(self.circuit.draw())

    # Reward function: we simulate our current state, take its resulting unitary matrix, and calculate the reward
    def _reward(self, target_unitary):
        simulator = Aer.get_backend('unitary_simulator')
        result = simulator.run(transpile(self.circuit, simulator)).result()
        unitary = result.get_unitary(self.circuit)

        unitary_array = np.asarray(unitary)
        target_unitary_array = np.asarray(target_unitary)

        fidelity = np.abs(np.trace(unitary_array.conj().T @ target_unitary_array)) / (2 ** self.num_qubits)

        reward = -5 * self.circuit.size()
        done = False
        if fidelity > 0.99:
            done = True
            reward += 100
        return reward, done

    def close(self):
        pass

In [4]:
# Construct our environment
environment = QuantumEnv()

<a id='agent'></a>
## 4. Agent

Now, we need to create our Q-learning agent that will interact with the **QuantumEnv** environment. We initialize the agent with: \
**state size**: state space \
**action size**: action space \
**alpha**: learning rate \
**gamma**: discount factor \
**epsilon**: exploration rate \
**decay rate**: how quickly epsilon decreases over time \
**epsilon min**: minimum exploration rate can decay to \
**Q-table**: stores Q-values (expected return) for each state-action pair. Initially all zeros.

The Q-values are updated via the Bellman equation:
\begin{equation}
Q^{\text{new}}(S_t, A_t) \leftarrow \underbrace{(1 - \alpha)}_{\textstyle\text{learning rate}} \cdot \underbrace{Q(S_t, A_t)}_{\textstyle\text{current value}} + \underbrace{\alpha}_{\textstyle\text{learning rate}} \cdot \left( \underbrace{R_{t+1}}_{\textstyle\text{reward}} + \underbrace{\gamma}_{\textstyle\text{discount factor}} \cdot \underbrace{\max_{a} Q(S_{t+1}, a)}_{\textstyle\text{estimate of optimal future value}} \right).
\end{equation}

In [5]:
# Define the Q-learning agent
class QLearningAgent:
    # Initialize the agent's parameters
    def __init__(self, state_size, action_size, alpha, gamma, epsilon, decay_rate, epsilon_min):
        self.state_size = state_size
        self.action_size = action_size
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.epsilon_min = epsilon_min
        # Initialize the Q-table with zeros
        self.q_table = np.zeros((state_size, action_size))

    # Chooses an action in the action space
    def choose_action(self, state_index, train):
        if np.random.rand() < self.epsilon and train:
            # Exploration: random action
            action = np.random.randint(self.action_size)
        else:
            # Exploitation: choose the best action
            action = np.argmax(self.q_table[state_index])
        
        possible_actions = [
            [HGate(), [0]],
            [HGate(), [1]],
            [CXGate(), [0, 1]],
            [CXGate(), [1, 0]],
            [SGate(), [0]],
            [SGate(), [1]],
            [TGate(), [0]],
            [TGate(), [1]],
            [XGate(), [0]],
            [XGate(), [1]],
            [YGate(), [0]],
            [YGate(), [1]],
            [ZGate(), [0]],
            [ZGate(), [1]],
        ]
        
        return possible_actions[action],action

    # Update Q-value using the Bellman equation
    def update_q_table(self, state_index, action, reward, next_state_index):
        # Update the Q-table based on the agent's experience
        self.q_table[state_index, action] += self.alpha * (
            reward + self.gamma * np.max(self.q_table[next_state_index]) - self.q_table[state_index, action]
        )

    # Decreases the epsilon value
    def decay_exploration(self):
        self.epsilon = max(self.epsilon_min, self.epsilon * self.decay_rate)

In [6]:
# Construct our agent with predefined parameters
agent = QLearningAgent(state_size=100, action_size=14, alpha=0.5, gamma=0.95, epsilon=0.2, decay_rate=0.99, epsilon_min=0.01)

<a id='train'></a>
## 5. Training our Agent
We specify $500$ episodes and $500$ steps per episode in the training process. For each step, we have the agent choose an action to take, obtain the reward from its interaction with the environment, and update its Q-table.

In [7]:
# Train the agent
def train_agent(agent, environment, episodes, max_steps_per_episode):
    # Iterates through each episode
    for episode in range(episodes):
        # Reset the environment at the beginning of each episode
        state_index = environment.reset()
        episode_reward = 0
        # Iterates through number of steps per episode
        for step in range(max_steps_per_episode):
            # Choose an action
            action, action_index = agent.choose_action(state_index, train=True)
            
            # Take the action and observe the outcome
            next_state_index, reward, done = environment.step(action[0],action[1])
            episode_reward += reward 
            # Update the Q-table
            agent.update_q_table(state_index, action_index, reward, next_state_index)
            
            # Update the state
            state_index = next_state_index
            
            # Check if the episode is done
            if done:
                print("Generated circuit:")
                environment.render()
                print(f"Episode {episode + 1}: Total Reward = {episode_reward}")
                break
            if environment.circuit.size() > 10:
                episode_reward -= 100  # Negative reward for exceeding maximum gates
                break
        
        # Decay the exploration rate
        # Save results every 100 attempts
        if (episode + 1) % 100 == 0:
            print(f"Episode {episode + 1}: Total Reward = {episode_reward}")
        agent.decay_exploration()

In [8]:
# Train our agent
np.random.seed(1)
train_agent(agent, environment, episodes=500, max_steps_per_episode=500)

Episode 100: Total Reward = -430
Episode 200: Total Reward = -430
Generated circuit:
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 205: Total Reward = 85
Generated circuit:
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 207: Total Reward = 85
Generated circuit:
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 208: Total Reward = 85
Generated circuit:
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 209: Total Reward = 85
Generated circuit:
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 210: Total Reward = 85
Generated circuit:
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 211: Total Reward = 85
Generated circuit:
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 212: Total Reward = 85
Generated circuit:
     

<a id='test'></a>
## 6. Testing
We specify the same number of episodes as the training process. However, we do not update the agent's Q-table with the reward from its chosen action and always choose the path with the greatest Q-values.

In [9]:
# Test the agent
def test_agent(agent, environment, episodes, max_steps_per_episode):
    results = []
    for episode in range(episodes):
        # Reset the environment
        state_index = environment.reset()
        episode_reward = 0
        for step in range(max_steps_per_episode):
            # Choose an action (exploitation only, no exploration)
            action, action_index = agent.choose_action(state_index, train=False)
            
            # Take the action and observe the outcome
            next_state_index, reward, done = environment.step(action[0], action[1])
            
            # Update the state
            state_index = next_state_index
            
            # Check if the episode is done
            if done:
                episode_reward = reward
                results.append(1)
                break
        if episode_reward < 0:
            results.append(0)
        print(f"Episode {episode + 1}: Total Reward = {episode_reward}")
        environment.render()
    results = np.array(results)
    correct = np.sum(results)
    print(f"Correctly constructed circuit for {correct} episodes out of {len(results)} episodes")

In [10]:
# Test our agent
test_agent(agent, environment, episodes=500, max_steps_per_episode=500)

Episode 1: Total Reward = 90
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 2: Total Reward = 90
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 3: Total Reward = 90
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 4: Total Reward = 90
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 5: Total Reward = 90
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 6: Total Reward = 90
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 7: Total Reward = 90
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 8: Total Reward = 90
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 9: Total Reward = 90
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
Episode 10: Total R